# MessageBuilder HelloWorld Demonstration

This notebook demonstrates the **MessageBuilder** system - a fluent interface for building Converse API messages. This provides a more intuitive and type-safe way to construct messages compared to manual dictionary building.

## Key Features

- 🔧 **Fluent Interface**: Chain methods for intuitive message building
- 🧩 **Type Safety**: Enum-based formats and validation
- 🎯 **Auto-Detection**: Automatic format detection for media files
- 📝 **Easy Text**: Simple text content addition
- 🖼️ **Image Support**: Streamlined image content handling
- 📄 **Document Support**: Document processing with format detection
- 🎥 **Video Support**: Video content integration
- ✅ **Validation**: Built-in content validation and size limits

## Comparison with Manual Approach

**Manual approach:**
```python
messages = [
    {
        "role": "user",
        "content": [
            {"text": "Hello!"}
        ]
    }
]
```

**MessageBuilder approach:**
```python
message = create_user_message() \
    .add_text("Hello!") \
    .build()
messages = [message]
```

## Setup and Imports

In [ ]:
import sys
import json
from pathlib import Path
import logging
from datetime import datetime
from PIL import Image
import io

# Add the src directory to path for imports
sys.path.append(str(Path.cwd().parent / "src"))

# Import the LLMManager and related classes
from bestehorn_llmmanager.llm_manager import LLMManager
from bestehorn_llmmanager.bedrock.models.llm_manager_structures import AuthConfig, RetryConfig, AuthenticationType, RetryStrategy
from bestehorn_llmmanager.bedrock.exceptions.llm_manager_exceptions import LLMManagerError, ConfigurationError, AuthenticationError

# Import MessageBuilder components
from bestehorn_llmmanager import create_user_message, create_assistant_message, create_message
from bestehorn_llmmanager.message_builder_enums import RolesEnum, ImageFormatEnum, DocumentFormatEnum, VideoFormatEnum

# Configure logging for better visibility
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("✅ Imports successful!")
print(f"📁 Working directory: {Path.cwd()}")
print("🔧 MessageBuilder system imported and ready!")

## Helper Functions

In [ ]:
def read_image_bytes(image_path: str, max_size_mb: float = 5.0) -> bytes:
    """Read an image file as raw bytes for Bedrock API."""
    path = Path(image_path)
    file_size_mb = path.stat().st_size / (1024 * 1024)
    
    if file_size_mb > max_size_mb:
        print(f"⚠️  Image {path.name} is {file_size_mb:.2f}MB, attempting to resize...")
        return resize_image_to_limit(image_path, max_size_mb)
    
    with open(image_path, "rb") as image_file:
        return image_file.read()

def resize_image_to_limit(image_path: str, max_size_mb: float) -> bytes:
    """Resize an image to fit within the size limit."""
    with Image.open(image_path) as img:
        if img.mode in ('RGBA', 'P'):
            img = img.convert('RGB')
        
        quality = 85
        scale_factor = 0.9
        
        while True:
            buffer = io.BytesIO()
            img.save(buffer, format='JPEG', quality=quality, optimize=True)
            img_bytes = buffer.getvalue()
            
            size_mb = len(img_bytes) / (1024 * 1024)
            
            if size_mb <= max_size_mb:
                print(f"✅ Resized image to {size_mb:.2f}MB")
                return img_bytes
            
            if quality > 50:
                quality -= 10
            else:
                new_width = int(img.width * scale_factor)
                new_height = int(img.height * scale_factor)
                img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
                quality = 85
                
                if new_width < 100 or new_height < 100:
                    raise ValueError(f"Cannot resize image small enough to fit {max_size_mb}MB limit")

def read_video_bytes(video_path: str) -> bytes:
    """Read a video file as raw bytes for Bedrock API."""
    with open(video_path, "rb") as video_file:
        return video_file.read()

def display_response(response, title="Response"):
    """Display a formatted response from LLMManager."""
    print(f"\n{title}")
    print("=" * len(title))
    
    if response.success:
        print(f"✅ Success: {response.success}")
        print(f"🤖 Model: {response.model_used}")
        print(f"🌍 Region: {response.region_used}")
        print(f"🔗 Access Method: {response.access_method_used}")
        print(f"⏱️  Total Duration: {response.total_duration_ms:.2f}ms")
        if response.api_latency_ms:
            print(f"⚡ API Latency: {response.api_latency_ms:.2f}ms")
        
        content = response.get_content()
        if content:
            print(f"\n💬 Response Content:")
            print("-" * 20)
            print(content)
        
        usage = response.get_usage()
        if usage:
            print(f"\n📊 Token Usage:")
            for key, value in usage.items():
                print(f"   {key}: {value}")
    else:
        print(f"❌ Success: {response.success}")
        print(f"🔄 Attempts: {len(response.attempts)}")
    
    if response.warnings:
        print(f"\n⚠️  Warnings:")
        for warning in response.warnings:
            print(f"   - {warning}")

print("✅ Helper functions defined!")

## Initialize the LLMManager

In [ ]:
print("🚀 Initializing LLMManager...")

models = ["Claude 3.5 Sonnet v2", "Claude 3 Haiku", "Nova Pro", "Nova Lite"]
regions = ["us-east-1", "us-west-2", "eu-west-1"]

auth_config = AuthConfig(auth_type=AuthenticationType.PROFILE, profile_name="default")
retry_config = RetryConfig(max_retries=3, retry_strategy=RetryStrategy.REGION_FIRST)

try:
    manager = LLMManager(models=models, regions=regions, auth_config=auth_config, retry_config=retry_config, timeout=30)
    print(f"✅ LLMManager initialized successfully!")
    validation = manager.validate_configuration()
    print(f"   Valid: {'✅' if validation['valid'] else '❌'} {validation['valid']}")
except Exception as e:
    print(f"❌ Error initializing LLMManager: {e}")
    raise

## Example 1: Basic Text Conversation 💬

In [ ]:
print("💬 Example 1: Basic Text Conversation (MessageBuilder)")
print("=" * 50)

# Build the message using MessageBuilder fluent interface
message = create_user_message() \
    .add_text("Hello! Please introduce yourself and explain what you can help me with. Keep it concise but friendly.") \
    .build()

messages = [message]
print(f"🔧 Built message with role: {message['role']}, content blocks: {len(message['content'])}")

try:
    response = manager.converse(messages=messages, inference_config={"maxTokens": 500, "temperature": 0.7})
    display_response(response, "🗣️  Basic Text Response (MessageBuilder)")
except Exception as e:
    print(f"❌ Error in text conversation: {e}")

## Example 2: Image Analysis 🖼️

In [ ]:
print("🖼️  Example 2: Image Analysis (MessageBuilder)")
print("=" * 45)

eiffel_image_path = Path("../images/1200px-Tour_Eiffel_Wikimedia_Commons_(cropped).jpg")
if eiffel_image_path.exists():
    print(f"📸 Analyzing image: {eiffel_image_path.name}")
    
    try:
        image_bytes = read_image_bytes(str(eiffel_image_path))
        print(f"📦 Image size: {len(image_bytes) / 1024:.1f} KB")
        
        # Build the message using MessageBuilder with automatic format detection
        message = create_user_message() \
            .add_text("Please analyze this image in detail. What do you see? Describe the architecture, setting, and any notable features. Also identify what landmark this is.") \
            .add_image_bytes(bytes=image_bytes, filename=eiffel_image_path.name) \
            .build()
        
        messages = [message]
        print(f"🔧 Built message with {len(message['content'])} content blocks including image ({message['content'][1]['image']['format']})")
        
        response = manager.converse(messages=messages, inference_config={"maxTokens": 800, "temperature": 0.3})
        display_response(response, "🖼️  Image Analysis Response (MessageBuilder)")
        
    except Exception as e:
        print(f"❌ Error in image analysis: {e}")
else:
    print(f"❌ Image file not found: {eiffel_image_path}")

## Example 3: Multi-Image Comparison 🔍

In [ ]:
print("🔍 Example 3: Multi-Image Comparison (MessageBuilder)")
print("=" * 50)

eiffel_path = Path("../images/1200px-Tour_Eiffel_Wikimedia_Commons_(cropped).jpg")
tokyo_path = Path("../images/Tokyo_Tower_2023.jpg")

if eiffel_path.exists() and tokyo_path.exists():
    print(f"📸 Comparing images: {eiffel_path.name} and {tokyo_path.name}")
    
    try:
        eiffel_bytes = read_image_bytes(str(eiffel_path), max_size_mb=5.0)
        tokyo_bytes = read_image_bytes(str(tokyo_path), max_size_mb=5.0)
        
        # Build the message using MessageBuilder with method chaining for multiple images
        message = create_user_message() \
            .add_text("Please compare these two tower images. What are the similarities and differences? Identify both landmarks and discuss their architectural styles, historical significance, and visual characteristics.") \
            .add_image_bytes(bytes=eiffel_bytes, filename=eiffel_path.name) \
            .add_image_bytes(bytes=tokyo_bytes, filename=tokyo_path.name) \
            .build()
        
        messages = [message]
        print(f"🔧 Built message with {len(message['content'])} content blocks (text + 2 images)")
        
        response = manager.converse(messages=messages, inference_config={"maxTokens": 1000, "temperature": 0.4})
        display_response(response, "🔍 Image Comparison Response (MessageBuilder)")
        
    except Exception as e:
        print(f"❌ Error in image comparison: {e}")
else:
    print(f"❌ One or both image files not found")

## Example 4: Video Analysis 🎥

In [ ]:
print("🎥 Example 4: Video Analysis (MessageBuilder)")
print("=" * 42)

video_path = Path("../videos/EiffelTower.mp4")

if video_path.exists():
    print(f"🎬 Analyzing video: {video_path.name} ({video_path.stat().st_size / (1024*1024):.2f} MB)")
    
    try:
        video_bytes = read_video_bytes(str(video_path))
        
        # Build the message using MessageBuilder with automatic format detection
        message = create_user_message() \
            .add_text("Please analyze this video of the Eiffel Tower. Describe what you see happening in the video, the perspective/camera angle, lighting conditions, any movement or activity, and the overall atmosphere. What time of day does it appear to be?") \
            .add_video_bytes(bytes=video_bytes, filename=video_path.name) \
            .build()
        
        messages = [message]
        print(f"🔧 Built message with video ({message['content'][1]['video']['format']})")
        
        response = manager.converse(messages=messages, inference_config={"maxTokens": 1000, "temperature": 0.3})
        display_response(response, "🎥 Video Analysis Response (MessageBuilder)")
        
    except Exception as e:
        print(f"❌ Error in video analysis: {e}")
        if "size" in str(e).lower() or "limit" in str(e).lower():
            print("💡 This might be due to video file size limits.")
else:
    print(f"❌ Video file not found: {video_path}")

## Example 5: System Messages and Advanced Configuration 🔧

In [ ]:
print("🔧 Example 5: Advanced Configuration (MessageBuilder)")
print("=" * 47)

# System message to set behavior
system_messages = [
    {"text": "You are a professional tour guide specializing in French architecture and history. You provide detailed, accurate information with enthusiasm and expertise. Always include interesting historical facts and architectural details."}
]

# Build user message using MessageBuilder
message = create_user_message() \
    .add_text("Tell me about the Eiffel Tower's construction and why it was initially controversial among Parisians.") \
    .build()

messages = [message]
print(f"🔧 Built message for expert guide scenario")

try:
    response = manager.converse(
        messages=messages,
        system=system_messages,
        inference_config={"maxTokens": 600, "temperature": 0.4, "topP": 0.9, "stopSequences": ["END_OF_RESPONSE"]},
        request_metadata={"purpose": "educational_demo", "timestamp": datetime.now().isoformat()}
    )
    
    display_response(response, "🏛️  Expert Guide Response (MessageBuilder)")
    
except Exception as e:
    print(f"❌ Error in advanced configuration: {e}")

## Summary and Next Steps 📋

In [ ]:
print("📋 MessageBuilder HelloWorld Summary")
print("=" * 38)

print("\n✅ MessageBuilder Features Demonstrated:")
print("   🔧 Fluent Interface - Method chaining for intuitive message building")
print("   📝 Text Messages - Simple text content addition with validation")
print("   🖼️  Image Processing - Automatic format detection and validation")
print("   🔍 Multi-Image Support - Easy handling of multiple images in one message")
print("   🎥 Video Analysis - Streamlined video content integration")
print("   🎯 Auto-Detection - Intelligent format detection from filenames")
print("   ✅ Validation - Built-in content and size validation")
print("   🔧 Advanced Config - Compatible with system messages and custom settings")

print("\n🆚 MessageBuilder vs Manual Approach:")
print("   ✅ More intuitive and readable code")
print("   ✅ Automatic format detection")
print("   ✅ Built-in validation and error handling")
print("   ✅ Type safety with enums")
print("   ✅ Consistent API across content types")
print("   ✅ Reduced boilerplate code")
print("   ✅ Better maintainability")

print("\n🚀 Next Steps:")
print("   1. Use MessageBuilder in your projects for cleaner code")
print("   2. Explore the different format enums for explicit type control")
print("   3. Leverage auto-detection for rapid prototyping")
print("   4. Integrate with LLMManager for robust AI applications")
print("   5. Take advantage of built-in validation for safer code")
print("   6. Experiment with document processing capabilities")

print("\n🎉 MessageBuilder makes Converse API message building intuitive and safe!")
print("🔧 Enjoy the fluent interface and automatic validations!")